# Prediccion de Series de Tiempo (Time Series Forecasting)

Una serie de tiempo es una secuencia de datos ordenados cronologicamente. Por ejemplo: la temperatura diaria, el precio de una accion, la cantidad de pasajeros de una aerolinea por mes, etc.

En esta notebook aprenderemos:
1. Los tres componentes de una serie de tiempo
2. Como descomponer una serie en sus componentes
3. Como predecir valores futuros utilizando tres enfoques distintos:
   - **ARIMA**: el algoritmo clasico de series de tiempo
   - **LightGBM**: un algoritmo de Gradient Boosting adaptado para series de tiempo
   - **NHITS**: una red neuronal moderna disenada para series de tiempo

In [ ]:
!pip install statsmodels lightgbm neuralforecast -q

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

## Cargar y visualizar los datos

Utilizaremos el clasico dataset de **AirPassengers** que contiene la cantidad mensual de pasajeros de una aerolinea internacional entre 1949 y 1960.

In [ ]:
url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/airline-passengers.csv'
df = pd.read_csv(url)
df.columns = ['fecha', 'pasajeros']
df['fecha'] = pd.to_datetime(df['fecha'])
df = df.set_index('fecha')
print(f"Cantidad de observaciones: {len(df)}")
print(f"Periodo: {df.index[0].strftime('%Y-%m')} a {df.index[-1].strftime('%Y-%m')}")
df.head(10)

In [ ]:
df.plot(figsize=(12, 4), title='Pasajeros de aerolinea por mes')
plt.ylabel('Pasajeros (miles)')
plt.show()

Podemos observar que la cantidad de pasajeros crece con el tiempo y que cada anio se repite un patron similar. Esto nos lleva a hablar de los tres componentes fundamentales de una serie de tiempo.

## Los tres componentes de una serie de tiempo

Toda serie de tiempo se puede descomponer en tres componentes fundamentales:

### 1. Tendencia (Trend)
Es la direccion general a largo plazo de los datos. Puede ser creciente, decreciente o constante. En nuestro ejemplo, la cantidad de pasajeros tiene una clara **tendencia creciente** a lo largo de los anios.

### 2. Estacionalidad (Seasonality)
Son patrones que se repiten de forma regular en intervalos fijos de tiempo. En nuestro ejemplo, cada anio se repite un patron similar: **mas pasajeros en verano y menos en invierno**.

### 3. Residuo (Residual)
Es lo que queda despues de eliminar la tendencia y la estacionalidad. Representa **variaciones aleatorias o ruido** que no se puede explicar con los otros dos componentes.

La descomposicion puede ser:
- **Aditiva**: Serie = Tendencia + Estacionalidad + Residuo (cuando la variacion estacional es constante)
- **Multiplicativa**: Serie = Tendencia x Estacionalidad x Residuo (cuando la variacion estacional crece con la tendencia)

En nuestro caso, como la amplitud de las oscilaciones crece con el tiempo, usaremos descomposicion **multiplicativa**.

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

result = seasonal_decompose(df['pasajeros'], model='multiplicative', period=12)

fig = result.plot()
fig.set_size_inches(12, 8)
plt.tight_layout()
plt.show()

Podemos ver claramente los tres componentes:
- **Tendencia (Trend)**: crecimiento continuo de pasajeros
- **Estacionalidad (Seasonal)**: un patron que se repite cada 12 meses con picos en julio/agosto
- **Residuo (Resid)**: variaciones aleatorias que no siguen ningun patron

## Preparar datos de entrenamiento y prueba

En series de tiempo, la division de datos **siempre debe ser cronologica**: entrenamos con el pasado y evaluamos con el futuro. Nunca se mezclan datos aleatoriamente como en otros problemas de Machine Learning.

Usaremos los datos hasta diciembre de 1959 para entrenar y los 12 meses de 1960 para evaluar.

In [ ]:
train = df[df.index < '1960-01-01']
test = df[df.index >= '1960-01-01']

print(f"Entrenamiento: {len(train)} observaciones ({train.index[0].strftime('%Y-%m')} a {train.index[-1].strftime('%Y-%m')})")
print(f"Prueba: {len(test)} observaciones ({test.index[0].strftime('%Y-%m')} a {test.index[-1].strftime('%Y-%m')})")

plt.figure(figsize=(12, 4))
plt.plot(train.index, train['pasajeros'], label='Entrenamiento')
plt.plot(test.index, test['pasajeros'], label='Prueba', color='orange')
plt.legend()
plt.title('Division cronologica de los datos')
plt.show()

Crearemos una funcion para evaluar las metricas de cada modelo:

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

def evaluar_modelo(nombre, real, prediccion):
    mae = mean_absolute_error(real, prediccion)
    rmse = np.sqrt(mean_squared_error(real, prediccion))
    mape = np.mean(np.abs((real - prediccion) / real)) * 100
    print(f"{nombre}:")
    print(f"  MAE  = {mae:.2f}")
    print(f"  RMSE = {rmse:.2f}")
    print(f"  MAPE = {mape:.2f}%")
    return {'modelo': nombre, 'MAE': round(mae, 2), 'RMSE': round(rmse, 2), 'MAPE': round(mape, 2)}

resultados = []

## Modelo 1: ARIMA

[ARIMA](https://es.wikipedia.org/wiki/Modelo_autorregresivo_integrado_de_media_m%C3%B3vil) (AutoRegressive Integrated Moving Average) es el algoritmo clasico para series de tiempo. Tiene tres parametros principales:

- **p** (AutoRegressive): cuantos valores pasados se usan para predecir el siguiente
- **d** (Integrated): cuantas veces se diferencia la serie para hacerla estacionaria
- **q** (Moving Average): cuantos errores pasados se usan para corregir la prediccion

Para datos con estacionalidad usamos **SARIMA** que agrega parametros estacionales **(P, D, Q, m)** donde **m** es el periodo estacional (12 para datos mensuales).

In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX

model_arima = SARIMAX(train['pasajeros'], 
                       order=(1, 1, 1), 
                       seasonal_order=(1, 1, 1, 12))
model_arima_fit = model_arima.fit(disp=False)

pred_arima = model_arima_fit.forecast(steps=12)
pred_arima.index = test.index

r = evaluar_modelo('ARIMA', test['pasajeros'].values, pred_arima.values)
resultados.append(r)

In [ ]:
plt.figure(figsize=(12, 4))
plt.plot(train.index, train['pasajeros'], label='Entrenamiento')
plt.plot(test.index, test['pasajeros'], label='Real', marker='o')
plt.plot(test.index, pred_arima, label='ARIMA', marker='x', linestyle='--')
plt.legend()
plt.title('Prediccion con ARIMA')
plt.show()

## Modelo 2: Gradient Boosting con LightGBM

Los algoritmos de Gradient Boosting como [LightGBM](https://lightgbm.readthedocs.io/) no estan disenados especificamente para series de tiempo, pero podemos adaptarlos creando variables (features) a partir de la serie temporal:

- **Valores pasados (lags)**: el valor de hace 1 mes, 2 meses, ..., 12 meses
- **Variables de fecha**: mes del anio
- **Estadisticas moviles**: media movil de los ultimos 12 meses

Este proceso de crear variables se llama **feature engineering** y es clave para que estos algoritmos funcionen bien con series de tiempo.

In [ ]:
def crear_features(dataframe, lags=12):
    df_feat = dataframe.copy()
    df_feat['mes'] = df_feat.index.month
    df_feat['anio'] = df_feat.index.year
    for i in range(1, lags + 1):
        df_feat[f'lag_{i}'] = df_feat['pasajeros'].shift(i)
    df_feat['media_movil_12'] = df_feat['pasajeros'].shift(1).rolling(12).mean()
    df_feat = df_feat.dropna()
    return df_feat

df_features = crear_features(df)
print(f"Features creados: {list(df_features.columns)}")
df_features.head()

In [ ]:
import lightgbm as lgb

train_feat = df_features[df_features.index < '1960-01-01']
test_feat = df_features[df_features.index >= '1960-01-01']

X_train_lgb = train_feat.drop(columns=['pasajeros'])
y_train_lgb = train_feat['pasajeros']
X_test_lgb = test_feat.drop(columns=['pasajeros'])
y_test_lgb = test_feat['pasajeros']

model_lgb = lgb.LGBMRegressor(n_estimators=200, learning_rate=0.1, 
                               num_leaves=31, random_state=42, verbose=-1)
model_lgb.fit(X_train_lgb, y_train_lgb)
pred_lgb = model_lgb.predict(X_test_lgb)

r = evaluar_modelo('LightGBM', y_test_lgb.values, pred_lgb)
resultados.append(r)

In [ ]:
plt.figure(figsize=(12, 4))
plt.plot(train.index, train['pasajeros'], label='Entrenamiento')
plt.plot(test.index, test['pasajeros'], label='Real', marker='o')
plt.plot(test.index, pred_lgb, label='LightGBM', marker='x', linestyle='--')
plt.legend()
plt.title('Prediccion con LightGBM')
plt.show()

Podemos ver cuales features fueron mas importantes para el modelo:

In [ ]:
importancia = pd.DataFrame({
    'feature': X_train_lgb.columns,
    'importancia': model_lgb.feature_importances_
}).sort_values('importancia', ascending=True)

importancia.plot(x='feature', y='importancia', kind='barh', figsize=(8, 5), legend=False)
plt.title('Importancia de features en LightGBM')
plt.xlabel('Importancia')
plt.tight_layout()
plt.show()

## Modelo 3: Red Neuronal NHITS

[NHITS](https://arxiv.org/abs/2201.12886) (Neural Hierarchical Interpolation for Time Series) es una red neuronal moderna disenada especificamente para prediccion de series de tiempo. Es una evolucion de [N-BEATS](https://arxiv.org/abs/1905.10437) que utiliza interpolacion jerarquica para capturar patrones a diferentes escalas temporales.

Utilizaremos la libreria [NeuralForecast](https://nixtla.github.io/neuralforecast/) de Nixtla que nos permite usar redes neuronales avanzadas de forma sencilla.

La libreria requiere un formato especifico con tres columnas:
- **unique_id**: identificador de la serie
- **ds**: fecha
- **y**: valor a predecir

In [ ]:
from neuralforecast import NeuralForecast
from neuralforecast.models import NHITS

df_nf = df.reset_index()
df_nf.columns = ['ds', 'y']
df_nf['unique_id'] = 'pasajeros'
df_nf = df_nf[['unique_id', 'ds', 'y']]

train_nf = df_nf[df_nf['ds'] < '1960-01-01']

horizon = 12

model_nhits = NeuralForecast(
    models=[NHITS(h=horizon, input_size=24, max_steps=300)],
    freq='MS'
)
model_nhits.fit(df=train_nf)
pred_nhits_df = model_nhits.predict()

pred_nhits = pred_nhits_df['NHITS'].values

r = evaluar_modelo('NHITS', test['pasajeros'].values, pred_nhits)
resultados.append(r)

In [ ]:
plt.figure(figsize=(12, 4))
plt.plot(train.index, train['pasajeros'], label='Entrenamiento')
plt.plot(test.index, test['pasajeros'], label='Real', marker='o')
plt.plot(test.index, pred_nhits, label='NHITS', marker='x', linestyle='--')
plt.legend()
plt.title('Prediccion con NHITS (Red Neuronal)')
plt.show()

## Comparacion de modelos

Comparemos los tres modelos en un mismo grafico y en una tabla de metricas:

In [ ]:
plt.figure(figsize=(12, 5))
plt.plot(train.index[-36:], train['pasajeros'][-36:], label='Entrenamiento', color='gray')
plt.plot(test.index, test['pasajeros'], label='Real', marker='o', color='black', linewidth=2)
plt.plot(test.index, pred_arima, label='ARIMA', marker='s', linestyle='--')
plt.plot(test.index, pred_lgb, label='LightGBM', marker='^', linestyle='--')
plt.plot(test.index, pred_nhits, label='NHITS', marker='d', linestyle='--')
plt.legend()
plt.title('Comparacion de modelos')
plt.ylabel('Pasajeros (miles)')
plt.show()

print("\nResumen de metricas:")
df_resultados = pd.DataFrame(resultados)
print(df_resultados.to_string(index=False))

Cada enfoque tiene sus ventajas:
- **ARIMA**: facil de interpretar, funciona bien con pocas datos, ideal para series con patrones claros
- **LightGBM**: muy flexible, permite agregar variables externas facilmente, rapido de entrenar
- **NHITS**: puede capturar patrones complejos automaticamente, muy potente con grandes cantidades de datos

## Ejercicio 1:

Prueba diferentes parametros para ARIMA. Modifica los valores de (p, d, q) y (P, D, Q, 12) y compara los resultados. Puedes probar por ejemplo:
- order=(2, 1, 2), seasonal_order=(1, 1, 1, 12)
- order=(1, 1, 1), seasonal_order=(2, 1, 2, 12)
- order=(0, 1, 1), seasonal_order=(0, 1, 1, 12)

In [ ]:
# Escribir aqui la solucion



In [ ]:
#@title Solucion Ejercicio 1 {display-mode:"form"}

parametros = [
    ((1, 1, 1), (1, 1, 1, 12)),
    ((2, 1, 2), (1, 1, 1, 12)),
    ((1, 1, 1), (2, 1, 2, 12)),
    ((0, 1, 1), (0, 1, 1, 12)),
]

for order, seasonal_order in parametros:
    model = SARIMAX(train['pasajeros'], order=order, seasonal_order=seasonal_order)
    model_fit = model.fit(disp=False)
    pred = model_fit.forecast(steps=12)
    mae = mean_absolute_error(test['pasajeros'].values, pred.values)
    rmse = np.sqrt(mean_squared_error(test['pasajeros'].values, pred.values))
    print(f"ARIMA{order} x {seasonal_order}: MAE={mae:.2f}, RMSE={rmse:.2f}")

## Ejercicio 2:

Reemplaza LightGBM por [XGBoost](https://xgboost.readthedocs.io/). Instala xgboost con `!pip install xgboost -q` y utiliza `XGBRegressor` con los mismos features que creamos anteriormente. Compara los resultados con LightGBM.

In [ ]:
# Escribir aqui la solucion



In [ ]:
#@title Solucion Ejercicio 2 {display-mode:"form"}

!pip install xgboost -q
from xgboost import XGBRegressor

model_xgb = XGBRegressor(n_estimators=200, learning_rate=0.1, random_state=42, verbosity=0)
model_xgb.fit(X_train_lgb, y_train_lgb)
pred_xgb = model_xgb.predict(X_test_lgb)

evaluar_modelo('XGBoost', y_test_lgb.values, pred_xgb)

plt.figure(figsize=(12, 4))
plt.plot(test.index, test['pasajeros'], label='Real', marker='o', color='black')
plt.plot(test.index, pred_lgb, label='LightGBM', marker='^', linestyle='--')
plt.plot(test.index, pred_xgb, label='XGBoost', marker='s', linestyle='--')
plt.legend()
plt.title('LightGBM vs XGBoost')
plt.show()

# Fin: [Volver al contenido del curso](https://www.freecodingtour.com/cursos/espanol/datascience/datascience.html)